# Automated AI Sales Email Generation System

## 📖 Overview
This notebook demonstrates a multi-agent AI system for automating the creation and sending of cold sales emails for a fictional company, **Pet Vet**. 

The workflow is as follows:
1.  **Draft Generation**: Three distinct AI agents, each with a unique personality (professional, humorous, concise), generate email drafts.
2.  **Selection**: A `Senior Sales Manager` agent reviews the drafts and selects the most promising one.
3.  **Formatting & Sending**: The selected draft is handed off to an `Email Manager` agent, which generates a subject line, converts the body to HTML, and sends the final email using the SendGrid API.

## 1. Setup and Imports

First, we import the necessary libraries. Ensure you have a `requirements.txt` file and have run `pip install -r requirements.txt`.

In [1]:
import os
import json
import asyncio
from typing import Dict

from dotenv import load_dotenv
from openai import AsyncOpenAI
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

# Assuming 'agents' is a custom library as implied by the original code
# You would need to have this library available in your environment.
# pip install agents-dev
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel

## 2. API Configuration

We load API keys from a `.env` file for security. This allows us to keep credentials out of the source code. The code also verifies that the keys have been loaded successfully.

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_GEMINI_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
sendgrid_api_key = os.getenv('SENDGRID_API_KEY')

api_keys = {
    "OpenAI": openai_api_key,
    "Google Gemini": google_api_key,
    "DeepSeek": deepseek_api_key,
    "Groq": groq_api_key,
    "SendGrid": sendgrid_api_key
}

for name, key in api_keys.items():
    if key:
        print(f"✅ {name} API Key is loaded.")
    else:
        print(f"❌ {name} API Key is missing or not working.")

✅ OpenAI API Key is loaded.
✅ Google Gemini API Key is loaded.
✅ DeepSeek API Key is loaded.
✅ Groq API Key is loaded.
❌ SendGrid API Key is missing or not working.


## 3. Model and Client Initialization

Here, we define the base URLs for the different model providers and initialize asynchronous clients for each. We then wrap them in the `OpenAIChatCompletionsModel` class from the `agents` library for a consistent interface.

In [3]:
# Base URLs for various model APIs
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

# Initialize clients for each service
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)

# Define the models using a standardized interface
# Note: The model names should be checked against the provider's documentation.
# For Gemini, the format might be 'models/gemini-1.5-flash-latest'
gemini_model = OpenAIChatCompletionsModel(model="models/gemini-1.5-flash-latest", openai_client=gemini_client)
llama3_model = OpenAIChatCompletionsModel(model="llama3-70b-8192", openai_client=groq_client)
deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)

## 4. Agent Definitions

We define the core AI agents that will perform specific tasks. Each agent is given a name, a set of instructions (its persona and goal), and an LLM to power its reasoning.

### 4.1. Sales Agents (The Drafters)

These three agents are responsible for writing the initial email drafts. Each has a different style to provide a variety of options.

In [ ]:
instructions1 = """
You are a professional, serious sales agent for Pet Vet, a SaaS platform that simplifies scheduling, payments, and management for veterinary practices. 
Your task is to write a formal and respectful cold email.
"""

instructions2 = """
You are a humorous, engaging sales agent for Pet Vet, a SaaS platform for veterinary practices. 
Your task is to write a witty, engaging cold email that is memorable and likely to get a response.
"""

instructions3 = """
You are a busy, efficient sales agent for Pet Vet, a SaaS platform for veterinary practices. 
Your task is to write a concise, to-the-point cold email that respects the recipient's time.
"""

sales_agent1 = Agent(name="DeepSeek_Sales_Agent (Professional)", instructions=instructions1, model=deepseek_model)
sales_agent2 = Agent(name="Gemini_Sales_Agent (Humorous)", instructions=instructions2, model=gemini_model)
sales_agent3 = Agent(name="Llama3_Sales_Agent (Concise)", instructions=instructions3, model=llama3_model)

### 4.2. Formatting and Sending Agents (The Finishers)

These agents handle the final steps of preparing and sending the email. They are defined first so they can be passed as `handoffs` to the manager agent.

In [6]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Sends an HTML email with a given subject and body using SendGrid."""
    try:
        sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
        
        # IMPORTANT: Replace with your verified sender and desired recipient
        from_email = Email("your_verified_sender@example.com") 
        to_email = To("recipient@example.com")  
        
        content = Content("text/html", html_body)
        mail = Mail(from_email, to_email, subject, content)
        
        response = sg.client.mail.send.post(request_body=mail.get())
        
        if response.status_code == 222:
             return {"status": "success", "message": f"Email sent with status code {response.status_code}"}
        else:
             return {"status": "error", "message": f"Failed to send email. Status: {response.status_code}, Body: {response.body}"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

In [7]:
subject_instructions = "You are an expert copywriter. Your task is to write a compelling, short, and effective subject line for a given cold sales email body."
html_instructions = "You are a web designer. Your task is to convert a plain text email body (which may include markdown) into a clean, professional, and responsive HTML email body. Use simple styles."

subject_writer = Agent(name="Subject_Writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Writes a subject for a cold sales email.")

html_converter = Agent(name="HTML_Converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Converts a plain text email body to an HTML email body.")

email_formatter_instructions = """
You are an email formatting and sending specialist. Your process is:
1. Receive the final plain text body of an email.
2. Use the `subject_writer` tool to generate a subject line.
3. Use the `html_converter` tool to transform the plain text body into HTML.
4. Use the `send_html_email` tool to send the final email with the generated subject and HTML body.
"""

email_manager_tools = [subject_tool, html_tool, send_html_email]

emailer_agent = Agent(
    name="Email_Manager",
    instructions=email_formatter_instructions,
    tools=email_manager_tools,
    model="gpt-4o-mini",
    handoff_description="Formats and sends the final email."
)

### 4.3. Senior Sales Manager (The Orchestrator)

This agent manages the entire process. It uses the sales agents as tools to generate drafts, makes a final decision, and then hands off the selected email to the `emailer_agent` for final processing.

In [ ]:
description = "Use this tool to write a cold sales email draft."
tool1 = sales_agent1.as_tool(tool_name="professional_sales_agent", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="humorous_sales_agent", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="concise_sales_agent", tool_description=description)

manager_tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

senior_sales_manager_instructions = """
You are a Senior Sales Manager at Pet Vet. Your goal is to send the most effective cold sales email possible.

Follow these steps carefully:
1. **Generate Drafts**: Use all three sales agent tools (`professional_sales_agent`, `humorous_sales_agent`, `concise_sales_agent`) to generate three different email drafts based on the user's request.
2. **Evaluate and Select**: Review the three drafts and choose the single best email. Use your judgment to determine which one is most likely to be effective.
3. **Handoff for Sending**: After selecting the best email, hand it off to the `Email_Manager` agent to handle the final formatting and sending.

**Crucial Rules**:
- You must use the sales agent tools to generate the drafts—do not write them yourself.
- You must select only ONE email to handoff.
- You must use the handoff feature to pass the selected email to the `Email_Manager`.
"""

senior_sales_manager = Agent(
    name="Senior_Sales_Manager",
    instructions=senior_sales_manager_instructions,
    tools=manager_tools,
    handoffs=handoffs,
    model="gpt-4o-mini"
)

## 5. Execution

This is the final step where we run the entire system. We provide an initial message to the `senior_sales_manager` and the `Runner` executes the defined workflow. The `trace` context manager provides detailed logging of the agent's actions and thoughts.

In [ ]:
async def main():
    # The initial prompt that kicks off the entire process
    message = "Send a cold sales email addressed to 'Dear Clinic Director' from 'Alex at Pet Vet'."
    
    print(f"--- Starting Automated SDR Workflow ---")
    print(f"Initial Prompt: {message}")
    
    with trace("Automated_SDR"):
        result = await Runner.run(senior_sales_manager, message)
    
    print(f"--- Workflow Complete ---")
    print(f"Final Result: {result}")

# Run the asynchronous main function
# In a Jupyter environment, you might need to handle the event loop differently
# if you get a 'RuntimeError: This event loop is already running'.
await main()

--- Starting Automated SDR Workflow ---
Initial Prompt: Send a cold sales email addressed to 'Dear Clinic Director' from 'Alex at DaySmart Vet'.
--- Workflow Complete ---
Final Result: RunResult:
- Last agent: Agent(name="Email_Manager", ...)
- Final output (str):
    It looks like there was an issue while attempting to send the email. 
    
    To proceed, please confirm that you have the necessary credentials for sending the email or let me know if you'd like to provide the details again for the transaction.
- 30 new item(s)
- 8 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
